# Libraries

In [1]:
import pyedflib
import plotly.express as px
from pathlib import Path
import os
import pandas as pd
import numpy as np
import EDF_wrapper
import filters

In [2]:
directory = Path("data/edf/")
files = EDF_wrapper.read_files_from_dir(directory)
files

[{'filepath': 'data/edf/1-1-Bewegung_edited.bdf',
  'signals': array([[-2.65347050e+00, -4.09816558e+00, -4.09922416e+00, ...,
           1.10268599e-05,  1.10268599e-05,  1.10268599e-05],
         [-1.54233793e+00, -1.44602933e+00, -1.44344905e+00, ...,
           1.10268599e-05,  1.10268599e-05,  1.10268599e-05],
         [-3.29887007e+00,  4.98973220e+00,  4.99647129e+00, ...,
           7.58485959e-06,  7.58485959e-06,  7.58485959e-06],
         [-4.99413858e+00, -5.02268751e-02, -4.44226541e-02, ...,
           6.99207944e-06,  6.99207944e-06,  6.99207944e-06],
         [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
           0.00000000e+00,  0.00000000e+00,  0.00000000e+00]]),
  'signal_headers': [{'label': 'EMG 1',
    'dimension': 'mV',
    'sample_rate': 4000.0,
    'sample_frequency': 4000.0,
    'physical_max': 185.0,
    'physical_min': -185.0,
    'digital_max': 8388607,
    'digital_min': -8388608,
    'prefilter': '',
    'transducer': 'transkutan'},
   {'labe

In [21]:
def create_swallows_df(file, fileList=False):
    swallow = iter(filter(lambda x : "_" not in x[-1], file["header"]["annotations"]))

    rows = {"set": [], "filepath": [], "category": [], "sample": [], "start_time": [], "stop_time": []}

    for ann in swallow:
        start_time, _, desc = ann
        stop_time, _, _ = next(swallow)
        
        if not fileList:
            rows["set"].append(1)
            rows["filepath"].append(Path(file["filepath"]).name)
            rows["category"].append(None)
            rows["sample"].append(desc)
            rows["start_time"].append(start_time)
            rows["stop_time"].append(stop_time)

    df = pd.DataFrame(rows)
    
    df['duration'] = df["stop_time"] - df["start_time"]
    
    return df

In [37]:
def create_general_df(file, fileList=False):
    def find_first_element(list_data, condition):
        for element in list_data:
            if condition(element):
                return element
        return None

    general = list(filter(lambda x : "_" in x[-1], file["header"]["annotations"]))

    rows = {"set": [], "subject": [], "category": [], "sample": [], "start_time": [], "stop_time": []}

    for i, row in enumerate(general):
        time, _, desc = row
        s = desc.split("_")
        
        if s[0] == "C":
            if s[-1] == "start":
                _, cat, _ = s
            else:
                cat = None
            
        else:
            m, sample, event = s
            if event == "start":
                start_time = time
                stop_time, _, _ = find_first_element(general[i:], lambda x: x[-1] == f"{m}_{sample}_stop")
                if not fileList:
                    rows["set"].append(1)
                    rows["subject"].append(Path(file["filepath"]).name)
                    rows["category"].append(cat)
                    rows["sample"].append(s[1])
                    rows["start_time"].append(start_time)
                    rows["stop_time"].append(stop_time)
                else:
                    pass
            else:
                pass

    df = pd.DataFrame(rows)
    
    df['duration'] = df["stop_time"] - df["start_time"]
     
    return df

In [38]:
file = files[4]

In [39]:
file["signals"]

array([[-8.24463983e+00, -1.04011408e+01, -1.04016701e+01, ...,
         1.10268599e-05,  1.10268599e-05,  1.10268599e-05],
       [-3.03411770e+00, -2.93099451e+00, -2.93311166e+00, ...,
         1.10268599e-05,  1.10268599e-05,  1.10268599e-05],
       [ 1.55562506e+00,  5.28698009e+00,  5.28642312e+00, ...,
         6.43697479e-06,  6.43697479e-06,  6.43697479e-06],
       [-4.60055487e+00, -3.27341003e+00, -3.26253536e+00, ...,
         1.09085608e-05,  1.09085608e-05,  1.09085608e-05],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00]])

In [40]:
file["header"]["annotations"]

[[2.8425, -1.0, 'Schlucken normal'],
 [5.925, -1.0, 'C_category2_start'],
 [6.687, -1.0, 'P_elevation_start'],
 [6.687, -1.0, 'P_swallow_start'],
 [7.304, -1.0, 'P_elevation_stop'],
 [7.304, -1.0, 'P_lowering_start'],
 [7.539, -1.0, 'P_lowering_stop'],
 [7.539, -1.0, 'P_swallow_stop'],
 [8.5357, -1.0, 'stop'],
 [11.0007, -1.0, 'Schlucken normal'],
 [15.463, -1.0, 'P_elevation_start'],
 [15.463, -1.0, 'P_swallow_start'],
 [16.079, -1.0, 'P_elevation_stop'],
 [16.079, -1.0, 'P_lowering_start'],
 [16.719, -1.0, 'P_lowering_stop'],
 [16.719, -1.0, 'P_swallow_stop'],
 [18.3997, -1.0, 'stop'],
 [19.502, -1.0, 'C_category2_stop'],
 [20.3732, -1.0, 'Schlucken hoch'],
 [26.7485, -1.0, 'stop'],
 [28.8607, -1.0, 'Schlucken hoch'],
 [33.0822, -1.0, 'stop'],
 [34.8947, -1.0, 'Schlucken tief'],
 [40.2047, -1.0, 'stop'],
 [41.9152, -1.0, 'Schlucken tief'],
 [45.999, -1.0, 'stop'],
 [48.576, -1.0, 'Schlucken Knie re'],
 [56.2337, -1.0, 'stop'],
 [58.7792, -1.0, 'Schlucken Knie li'],
 [64.0495, -1.0, '

In [41]:
general = list(filter(lambda x : "_" in x[-1], file["header"]["annotations"]))
general

[[5.925, -1.0, 'C_category2_start'],
 [6.687, -1.0, 'P_elevation_start'],
 [6.687, -1.0, 'P_swallow_start'],
 [7.304, -1.0, 'P_elevation_stop'],
 [7.304, -1.0, 'P_lowering_start'],
 [7.539, -1.0, 'P_lowering_stop'],
 [7.539, -1.0, 'P_swallow_stop'],
 [15.463, -1.0, 'P_elevation_start'],
 [15.463, -1.0, 'P_swallow_start'],
 [16.079, -1.0, 'P_elevation_stop'],
 [16.079, -1.0, 'P_lowering_start'],
 [16.719, -1.0, 'P_lowering_stop'],
 [16.719, -1.0, 'P_swallow_stop'],
 [19.502, -1.0, 'C_category2_stop'],
 [83.653, -1.0, 'P_swallow_start'],
 [85.317, -1.0, 'P_swallow_stop'],
 [156.447, -1.0, 'P_movement_start'],
 [158.004, -1.0, 'P_movement_stop'],
 [162.616, -1.0, 'P_movement_start'],
 [163.881, -1.0, 'P_movement_stop']]

In [42]:
swallows_df = create_swallows_df(file)
swallows_df

,set,filepath,category,sample,start_time,stop_time,duration
0,1,data/edf/1-8-Bewegung_edited.bdf,None,Schlucken normal,2.8425,8.5357,5.6932
1,1,data/edf/1-8-Bewegung_edited.bdf,None,Schlucken normal,11.0007,18.3997,7.3990
2,1,data/edf/1-8-Bewegung_edited.bdf,None,Schlucken hoch,20.3732,26.7485,6.3753
3,1,data/edf/1-8-Bewegung_edited.bdf,None,Schlucken hoch,28.8607,33.0822,4.2215
4,1,data/edf/1-8-Bewegung_edited.bdf,None,Schlucken tief,34.8947,40.2047,5.3100
5,1,data/edf/1-8-Bewegung_edited.bdf,None,Schlucken tief,41.9152,45.9990,4.0838
6,1,data/edf/1-8-Bewegung_edited.bdf,None,Schlucken Knie re,48.5760,56.2337,7.6577
7,1,data/edf/1-8-Bewegung_edited.bdf,None,Schlucken Knie li,58.7792,64.0495,5.2703
8,1,data/edf/1-8-Bewegung_edited.bdf,None,Mendelson,67.5430,79.0050,11.4620
9,1,data/edf/1-8-Bewegung_edited.bdf,None,Mendelson,81.0500,86.7565,5.7065


In [43]:
general_df = create_general_df(file)
general_df

,set,subject,category,sample,start_time,stop_time,duration
0,1,1-8-Bewegung_edited.bdf,category2,elevation,6.687,7.304,0.617
1,1,1-8-Bewegung_edited.bdf,category2,swallow,6.687,7.539,0.852
2,1,1-8-Bewegung_edited.bdf,category2,lowering,7.304,7.539,0.235
3,1,1-8-Bewegung_edited.bdf,category2,elevation,15.463,16.079,0.616
4,1,1-8-Bewegung_edited.bdf,category2,swallow,15.463,16.719,1.256
5,1,1-8-Bewegung_edited.bdf,category2,lowering,16.079,16.719,0.640
6,1,1-8-Bewegung_edited.bdf,None,swallow,83.653,85.317,1.664
7,1,1-8-Bewegung_edited.bdf,None,movement,156.447,158.004,1.557
8,1,1-8-Bewegung_edited.bdf,None,movement,162.616,163.881,1.265


In [12]:
signal = file["signals"][0]
fs = file["signal_headers"][0]["sample_frequency"]

signal = filters.apply_butterworth(signal, fs, "high", 15, 3)

for notch_freq in [50, 100, 150]:
    signal = filters.apply_notch(signal, fs, notch_freq, Q=15)

px.line(signal[:80000])

# Adjust the physical values for the filtered signals

In [24]:
for signal, header in zip(file["signals"], file["signal_headers"]):
    for key, value in header.items():
        if key == "physical_max":
            header[key] = signal.max()
        if key == "physical_min":
            header[key] = signal.min()